In [1]:
import os
import sys
import itertools
import pickle
from glob import glob
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd
from scipy.stats import spearmanr

from matplotlib import pyplot as plt
from matplotlib_venn import venn2, venn3
import seaborn as sns

import torch
from torch import nn, optim
from transformers import BertConfig, BertTokenizer, BertModel, BertForMaskedLM#, BertLayer, BertEmbeddings
from transformers.modeling_bert import BertLayer, BertEmbeddings

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# re-load functions
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

In [3]:
DEVICE = 'cpu'

In [4]:
import sys
import pickle
from functools import partial
from glob import glob

import numpy as np
import pandas as pd
import scipy as sp
import torch
from scipy.stats import spearmanr
from tqdm import tqdm

class OptimizedRounder(object):
    """
    An optimizer for rounding thresholds
    to maximize Quadratic Weighted Kappa (QWK) score
    # https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
    """

    def __init__(self):
        self.coef_ = 0

    def _spearmanr_loss(self, coef, X, y, labels):
        """
        Get loss according to
        using current coefficients
        :param coef: A list of coefficients that will be used for rounding
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) +
                     [np.inf], labels=labels)

        # return -np.mean(spearmanr(y, X_p).correlation)
        return -spearmanr(y, X_p).correlation

    def fit(self, X, y, initial_coef):
        """
        Optimize rounding thresholds
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        labels = self.labels
        loss_partial = partial(self._spearmanr_loss, X=X, y=y, labels=labels)
        self.coef_ = sp.optimize.minimize(
            # loss_partial, initial_coef, method='Powell')
            loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        """
        Make predictions with specified thresholds
        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        labels = self.labels
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) +
                      [np.inf], labels=labels)
        # [np.inf], labels=[0, 1, 2, 3])

    def coefficients(self):
        """
        Return the optimized coefficients
        """
        return self.coef_['x']

    def set_labels(self, labels):
        self.labels = labels

In [5]:
# sys.path.append('../scripts/')
# from get_optR3 import compute_spearmanr, get_opt_y_pred

import os
import pickle
import sys
from functools import partial
from glob import glob

import numpy as np
import pandas as pd
import scipy as sp
import torch
from scipy.stats import spearmanr
from tqdm import tqdm


class histogramBasedCoefInitializer:
    def __init__(self):
        self.bins = None

    def fit(self, labels):
        self.bins = pd.Series(
            labels).value_counts().sort_index().cumsum().values
        return self

    def predict(self, preds):
        preds = sorted(preds)
        res_threshs = []
        if self.bins is None:
            raise Exception('plz fit at first.')
        for _bin in self.bins[:-1]:
            res_threshs.append((preds[_bin - 1] + preds[_bin]) / 2)
        return res_threshs


class OptimizedRounder(object):
    """
    An optimizer for rounding thresholds
    to maximize Quadratic Weighted Kappa (QWK) score
    # https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
    """

    def __init__(self):
        self.coef_ = 0

    def _spearmanr_loss(self, coef, X, y, labels):
        """
        Get loss according to
        using current coefficients
        :param coef: A list of coefficients that will be used for rounding
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) +
                     [np.inf], labels=labels)

        # return -np.mean(spearmanr(y, X_p).correlation)
        return -spearmanr(y, X_p).correlation

    def fit(self, X, y, initial_coef):
        """
        Optimize rounding thresholds
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        labels = self.labels
        loss_partial = partial(self._spearmanr_loss, X=X, y=y, labels=labels)
        self.coef_ = sp.optimize.minimize(
            loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        """
        Make predictions with specified thresholds
        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        labels = self.labels
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) +
                      [np.inf], labels=labels)
        # [np.inf], labels=[0, 1, 2, 3])

    def coefficients(self):
        """
        Return the optimized coefficients
        """
        return self.coef_['x']

    def set_labels(self, labels):
        self.labels = labels


def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        if len(np.unique(col_pred)) == 1:
            if col_pred[0] == np.max(col_trues):
                col_pred[np.argmin(
                    col_pred)] = np.min(col_trues)
            else:
                col_pred[np.argmax(
                    col_pred)] = np.max(col_trues)
        rhos.append(
            spearmanr(
                col_trues,
                col_pred
                #                  + np.random.normal(
                #                     0,
                #                     1e-7,
                #                     col_pred.shape[0])
            ).correlation)
    return rhos


def get_best_ckpt(ckpts):
    ckpt_dicts = []
    for ckpt in ckpts:
        ckpt_dict = {}
        ckpt_dict['ckpt'] = ckpt
        splitted_ckpt = ckpt.split('/')[-1].split('_')
        ckpt_dict['val_metric'] = float(splitted_ckpt[5])
        ckpt_dicts.append(ckpt_dict)
    ckpt_df = pd.DataFrame(ckpt_dicts)
    return ckpt_df.sort_values('val_metric', ascending=False).ckpt.iloc[0]


def get_snapshot_info_df(base_dir):
    res_dicts = []
    for fold in tqdm(list(range(5))):
        fold_ckpts = glob(f'{base_dir}/{fold}/*.pth')
        for ckpt in fold_ckpts:
            res_dict = {}
            splitted_ckpt = ckpt.split('/')[-1].split('_')
            res_dict['ckpt_filename'] = ckpt
            res_dict['fold'] = int(splitted_ckpt[1])
            res_dict['epoch'] = int(splitted_ckpt[3])
            res_dict['val_loss'] = float(splitted_ckpt[4])
            res_dict['val_metric'] = float(splitted_ckpt[5])
            res_dicts.append(res_dict)
    res_df = pd.DataFrame(res_dicts)
    res_df['rank'] = res_df.groupby(['fold']).val_metric.rank()
    return res_df


def get_opt_y_pred(y_true, y_pred, num_labels):
    optRs = []
    opt_y_preds = []

    # for i in tqdm(list(range(21))):
    for i in range(num_labels):
        optR = OptimizedRounder()
        labels = np.sort(np.unique(y_true[:, i]))
        optR.set_labels(labels)
        initer = histogramBasedCoefInitializer().fit(y_true[:, i])
        opt_thresh = initer.predict(y_pred[:, i])
        optR.fit(y_pred[:, i], y_true[:, i], opt_thresh)
        # opt_threshs.append(optR.coefficients())
        # opt_threshs[i] = optR.coefficients()
        optRs.append(optR)
        opt_y_preds.append((optR.predict(y_pred[:, i], optR.coefficients())))

    opt_y_preds = np.asarray(opt_y_preds).T
    return optRs, opt_y_preds


def opt(BASE_PATH, num_labels=30, snapshot_num=2):
    snapshot_df = get_snapshot_info_df(BASE_PATH)

    snapshot_dicts = {}
    state_dict_dicts = {}
    for fold in tqdm(list(range(5))):
        snapshot_dict = {}
        state_dict_dict = {}
        for i, row in snapshot_df.query(f'fold == {fold}').sort_values(
                'rank', ascending=False).reset_index(drop=True).iterrows():
            if i >= snapshot_num:
                continue
            ckpt = torch.load(row['ckpt_filename'])
            state_dict_dict[i] = ckpt['model_state_dict']
            if i == 0:
                qa_ids = ckpt['val_qa_ids']
                qa_ids_argsort = np.argsort(qa_ids)
                snapshot_dict['y_trues'] = [
                    ckpt['val_y_trues'][qa_ids_argsort]]
                snapshot_dict['y_preds'] = [
                    ckpt['val_y_preds'][qa_ids_argsort]]
            else:
                qa_ids = ckpt['val_qa_ids']
                qa_ids_argsort = np.argsort(qa_ids)
                snapshot_dict['y_trues'].append(
                    ckpt['val_y_trues'][qa_ids_argsort])
                snapshot_dict['y_preds'].append(
                    ckpt['val_y_preds'][qa_ids_argsort])
        snapshot_dicts[fold] = snapshot_dict
        state_dict_dicts[fold] = state_dict_dict

    y_preds = np.concatenate(
        [np.average(snapshot_dicts[fold]['y_preds'][:snapshot_num], axis=0)
         for fold in range(5)])
    y_trues = np.concatenate(
        [snapshot_dicts[fold]['y_trues'][0]
         for fold in range(5)])

    reses = []
    optRs = []

    for i in tqdm(list(range(num_labels))):
        y_pred = y_preds[:, i]
        y_true = y_trues[:, i]

        y_pred_argmax = np.argmax(y_pred)
        y_pred_argmin = np.argmin(y_pred)

        optR = OptimizedRounder()
        labels = np.sort(np.unique(y_true))
        optR.set_labels(labels)
        initer = histogramBasedCoefInitializer().fit(y_true)
        initial_coef = initer.predict(y_pred)
        optR.fit(y_pred, y_true, initial_coef=initial_coef)
        optRs.append(optR)
        res = optR.predict(y_pred, optR.coefficients())

        if len(np.unique(res)) == 1:
            if np.unique(res) == res[y_pred_argmax]:
                res[y_pred_argmin] = np.min(y_true)
            else:
                res[y_pred_argmax] = np.max(y_true)

        reses.append(res)
    reses = np.asarray(reses).T

    with open(f'{BASE_PATH}/optRs.pkl', 'wb') as fout:
        pickle.dump(optRs, fout)
    with open(f'{BASE_PATH}/snapshot_dicts.pkl', 'wb') as fout:
        pickle.dump(snapshot_dicts, fout)
    if not os.path.exists(f'{BASE_PATH}/state_dicts'):
        os.mkdir(f'{BASE_PATH}/state_dicts')
    for fold in range(5):
        for rank in range(snapshot_num):
            with open(f'{BASE_PATH}/state_dicts/fold_{fold}_rank_{rank}_state_dict.pkl', 'wb') as fout:
                pickle.dump(state_dict_dicts[fold][rank], fout)
    # with open(f'{BASE_PATH}/state_dict_dicts.pkl', 'wb') as fout:
    #     pickle.dump(state_dict_dicts, fout)

    original_score = compute_spearmanr(y_trues, y_preds)
    print(f'original_score: {original_score}')
    print(f'original_score: {np.mean(original_score)}')

    res_score = compute_spearmanr(y_trues, reses)
    print(f'res_score: {res_score}')
    print(f'res_score_mean: {np.mean(res_score)}')

    return res_score

In [6]:
class histogramBasedCoefInitializer:
    def __init__(self):
        self.bins = None

    def fit(self, labels):
        self.bins = pd.Series(labels).value_counts().sort_index().cumsum().values
        return self

    def predict(self, preds):
        preds = sorted(preds)
        res_threshs = []
        if self.bins is None:
            raise Exception('plz fit at first.')
        for _bin in self.bins[:-1]:
            res_threshs.append((preds[_bin - 1] + preds[_bin]) / 2)
        return res_threshs

In [16]:
def blend_and_evaluate(y_trues, y_preds_list, eval_func, weights=None):
    if weights:
        y_preds = np.average(y_preds_list, axis=0, weights=weights)
    else:
        y_preds = np.average(y_preds_list, axis=0)
    eval_scores = eval_func(y_trues, y_preds)
    optRs, opt_y_preds = get_opt_y_pred(y_trues, y_preds, num_labels=30)
    opt_eval_scores = eval_func(y_trues, opt_y_preds)
    print(f'original_score: {np.mean(eval_scores)}')
    print(f'opt_score: {np.mean(opt_eval_scores)}')
    return eval_scores, opt_eval_scores, optRs

## まずは top2 optRs を作る

## snapshot 済みの model を load

In [9]:
CKPT_DIR = '../mnt/checkpoints'

with open(f'{CKPT_DIR}/e078/snapshot_dicts.pkl', 'rb') as fin:
    bert_question_dict = pickle.load(fin)
with open(f'{CKPT_DIR}/e079/snapshot_dicts.pkl', 'rb') as fin:
    bert_answer_dict = pickle.load(fin)
    
with open(f'{CKPT_DIR}/e080/snapshot_dicts.pkl', 'rb') as fin:
    roberta_question_dict = pickle.load(fin)
with open(f'{CKPT_DIR}/e081/snapshot_dicts.pkl', 'rb') as fin:
    roberta_answer_dict = pickle.load(fin)
    
# with open(f'{CKPT_DIR}/e072/snapshot_dicts.pkl', 'rb') as fin:
#     gpt2_question_dict = pickle.load(fin)
# with open(f'{CKPT_DIR}/e073/snapshot_dicts.pkl', 'rb') as fin:
#     gpt2_answer_dict = pickle.load(fin)
    
with open(f'{CKPT_DIR}/e082/snapshot_dicts.pkl', 'rb') as fin:
    xlnet_question_dict = pickle.load(fin)
with open(f'{CKPT_DIR}/e083/snapshot_dicts.pkl', 'rb') as fin:
    xlnet_answer_dict = pickle.load(fin)

In [10]:
def _get_y_trues_and_y_preds_from_snapshot_dicts(snapshot_dicts, single, avg):
    y_trues, y_preds = [], []
    for fold in range(5):
        if single:
            y_trues.append(snapshot_dicts[fold]['y_trues'][0])
            y_preds.append(snapshot_dicts[fold]['y_preds'][0])
        else:
            if avg:
                y_trues.append(np.average(snapshot_dicts[fold]['y_trues'], axis=0))
                y_preds.append(np.average(snapshot_dicts[fold]['y_preds'], axis=0))
            else:
                y_trues.append(np.concatenate(snapshot_dicts[fold]['y_trues'], axis=1))
                y_preds.append(np.concatenate(snapshot_dicts[fold]['y_preds'], axis=1))
    y_trues = np.concatenate(y_trues)
    y_preds = np.concatenate(y_preds)
    return y_trues, y_preds

def get_y_trues_and_y_preds_from_QA_snapshota_dicts(Q_snapshot_dicts, A_snapshot_dicts, single=False, avg=True, model_num=2):
    q_y_trues, q_y_preds = _get_y_trues_and_y_preds_from_snapshot_dicts(Q_snapshot_dicts, single, avg)
    a_y_trues, a_y_preds = _get_y_trues_and_y_preds_from_snapshot_dicts(A_snapshot_dicts, single, avg)
    if avg:
        y_trues = np.concatenate([q_y_trues, a_y_trues], axis=1)
        y_preds = np.concatenate([q_y_preds, a_y_preds], axis=1)
    else:
        y_trues = np.concatenate([np.concatenate([q_y_trues[:, i*21:(i+1)*21], a_y_trues[:, i*9:(i+1)*9]], axis=1) for i in range(model_num)], axis=1)
        y_preds = np.concatenate([np.concatenate([q_y_preds[:, i*21:(i+1)*21], a_y_preds[:, i*9:(i+1)*9]], axis=1) for i in range(model_num)], axis=1)
        # y_preds = np.concatenate([q_y_preds[:, i*21:(i+1)*21] for i in range(model_num)] + [a_y_preds[:, i*9:(i+1)*9] for i in range(model_num)], axis=1)
    return y_trues, y_preds

In [11]:
# %debug
bert_y_trues, bert_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(bert_question_dict, bert_answer_dict)
roberta_y_trues, roberta_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(roberta_question_dict, roberta_answer_dict)
# gpt2_y_trues, gpt2_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(gpt2_question_dict, gpt2_answer_dict)
xlnet_y_trues, xlnet_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(xlnet_question_dict, xlnet_answer_dict)

In [12]:
# %debug
single_bert_y_trues, single_bert_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(bert_question_dict, bert_answer_dict, single=True)
single_roberta_y_trues, single_roberta_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(roberta_question_dict, roberta_answer_dict, single=True)
# single_gpt2_y_trues, single_gpt2_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(gpt2_question_dict, gpt2_answer_dict, single=True)
single_xlnet_y_trues, single_xlnet_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(xlnet_question_dict, xlnet_answer_dict, single=True)

In [13]:
# 良さそう
(bert_y_trues == roberta_y_trues).all(), (bert_y_trues == xlnet_y_trues).all()

(True, True)

In [17]:
blend_and_evaluate(single_bert_y_trues, [single_bert_y_preds,  ], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [single_roberta_y_preds, ], compute_spearmanr)
# blend_and_evaluate(single_bert_y_trues, [single_gpt2_y_preds, ], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [single_xlnet_y_preds], compute_spearmanr)
print('fini!')

original_score: 0.39658756843296844
opt_score: 0.4240074936181189


/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:2525: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


original_score: 0.3952086168898242
opt_score: 0.417814664868746
original_score: 0.39510187276130454
opt_score: 0.42243226674838147
fini!


In [18]:
blend_and_evaluate(single_bert_y_trues, [bert_y_preds,  ], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [roberta_y_preds, ], compute_spearmanr)
# blend_and_evaluate(single_bert_y_trues, [gpt2_y_preds, ], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [xlnet_y_preds], compute_spearmanr)
print('fini!')

original_score: 0.40437836760234475
opt_score: 0.4374389165251154
original_score: 0.40003269950864506
opt_score: 0.4242294402147104
original_score: 0.4032963844416211
opt_score: 0.427509974207624
fini!


In [19]:
eval_scores, opt_eval_scores, optRs = blend_and_evaluate(single_bert_y_trues, [bert_y_preds, roberta_y_preds, xlnet_y_preds], compute_spearmanr)

original_score: 0.42022642838382596
opt_score: 0.4540915450569295


In [22]:
with open('../mnt/inputs/pseudos/top2_e078_e079_e080_e081_e082_e083/optRs.pkl', 'wb') as fout:
    pickle.dump(optRs, fout)

## 関数設計 (path を設定すると model を load して prediction を返してくる)

In [28]:
sys.path.append('../scripts/')
from refactor.datasets import QUESTDataset
from refactor.models import BertModelForBinaryMultiLabelClassifier, RobertaModelForBinaryMultiLabelClassifier, XLNetModelForBinaryMultiLabelClassifier
from refactor.utils import test
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SequentialSampler

In [ ]:
def predict_from_ckpt(df, ckpt, loader, TOKENIZER_TYPE, DO_LOWER_CASE, T_MAX_LEN, Q_MAX_LEN, A_MAX_LEN, TQA_MODE):
    if TOKENIZER_TYPE == 'bert':
        state_dict = BertModel.from_pretrained('bert-base-uncased').state_dict()
        model = BertModelForBinaryMultiLabelClassifier(num_labels=, )
        ckpt = 
    dataset = QUESTDataset(
                                df=df,
                                mode='test',
                                tokens=[],
                                augment=[],
                                tokenizer_type=TOKENIZER_TYPE,
                                pretrained_model_name_or_path='../mnt/checkpoints/e078/datasets/',
                                do_lower_case=DO_LOWER_CASE,
                                LABEL_COL=LABEL_COL,
                                t_max_len=T_MAX_LEN,
                                q_max_len=Q_MAX_LEN,
                                a_max_len=A_MAX_LEN,
                                tqa_mode=TQA_MODE,
                                TBSEP='[TBSEP]',
                                pos_id_type='arange',
                                MAX_SEQUENCE_LENGTH=512,
                            )
    sampler = RandomSampler(data_source=dataset)
    loader = DataLoader(dataset,
                            batch_size=8,
                            sampler=sampler,
                            num_workers=os.cpu_count(),
                            worker_init_fn=lambda x: np.random.seed(),
                            drop_last=False,
                            pin_memory=True)    
    return prediction

#### まず Q 系を潰す

In [26]:
labels = [
    'question_asker_intent_understanding',
    'question_body_critical',
    'question_conversational',
    'question_expect_short_answer',
    'question_fact_seeking',
    'question_has_commonly_accepted_answer',
    'question_interestingness_others',
    'question_interestingness_self',
    'question_multi_intent',
    'question_not_really_a_question',
    'question_opinion_seeking',
    'question_type_choice',
    'question_type_compare',
    'question_type_consequence',
    'question_type_definition',
    'question_type_entity',
    'question_type_instructions',
    'question_type_procedure',
    'question_type_reason_explanation',
    'question_type_spelling',
    'question_well_written',
#     'answer_helpful',
#     'answer_level_of_information',
#     'answer_plausible',
#     'answer_relevance',
#     'answer_satisfaction',
#     'answer_type_instructions',
#     'answer_type_procedure',
#     'answer_type_reason_explanation',
#     'answer_well_written'
]

## feature engineering

In [293]:
trn_df = pd.read_csv('../mnt/inputs/origin/train.csv')

In [294]:
from sklearn.model_selection import GroupKFold as GKF

gkf = GKF(5)
fold = list(gkf.split(trn_df.qa_id, groups=trn_df.question_body))

In [295]:
val_idxes = [i[1] for i in fold]
val_idx = np.concatenate(val_idxes)

In [296]:
trn_df['fold'] = None
for i, _val_idx in enumerate(val_idxes):
    trn_df.loc[_val_idx, 'fold'] = i

In [297]:
trn_df['fold'].fillna(-1).value_counts()

3    1216
2    1216
1    1216
0    1216
4    1215
Name: fold, dtype: int64

In [298]:
(trn_df.iloc[val_idx].iloc[:, 11:-1].values == bert_y_trues[:, :30]).all()

True

In [299]:
trn_df = trn_df.iloc[val_idx].reset_index(drop=True)

In [301]:
# 置換完了
(trn_df.iloc[:, 11:-1].values == bert_y_trues[:, :30]).all()

True

In [442]:
from urllib.parse import urlparse
import re
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import MultiTaskElasticNet, Lasso, LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

import tqdm
tqdm.tqdm.pandas()
from tqdm import tqdm_notebook as tqdm

from transformers import BertTokenizer, RobertaTokenizer, XLNetTokenizer

def mk_features(df, bert_preds, roberta_preds, xlnet_preds,
                              bert_tokenizer_path, ohe=None, scaler=None, labels=[]):
    base_cols = ['qa_id', 'category', 'fold'] + labels
    features_df = df[base_cols]

    # categorize host
    HOSTs = ['stackexchange', 'askubuntu', 'mathoverflow', 'serverfault', 'stackoverflow', 'superuser']
    for HOST in HOSTs:
        features_df.loc[df.host.str.contains(HOST).values, 'host'] = f'HOST_{HOST}'.casefold()

    # url features
    MAP_DICT = {'cs': 'programmers', 'softwarerecs': 'programmers', 'robotics': 'electronics'}
    find = re.compile(r"^[^.]*")
    features_df['netloc'] = df['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0]).apply(lambda x: MAP_DICT[x] if x in MAP_DICT else x)
    
    # len types features
    features_df['title_len'] = df.question_title.apply(lambda x: len(x.casefold().split()))
    features_df['body_len'] = df.question_body.apply(lambda x: len(x.casefold().split()))
    features_df['answer_len'] = df.answer.apply(lambda x: len(x.casefold().split()))
    
    bert_tokenizer = BertTokenizer.from_pretrained(bert_tokenizer_path)
    features_df['bert_title_len'] = df.question_title.progress_apply(lambda x: len(bert_tokenizer.tokenize(x)))
    features_df['bert_body_len'] = df.question_body.progress_apply(lambda x: len(bert_tokenizer.tokenize(x)))
    features_df['bert_answer_len'] = df.answer.progress_apply(lambda x: len(bert_tokenizer.tokenize(x)))
    
    # res_ohes = []
    # for i, feature in enumerate(['category', 'host', 'netloc']):
    #     if not ohes:
    #         ohe = OneHotEncoder()
    #         ohe.fit(features_df[feature])
    #         res_ohes.append(ohe)
    #     else:
    #         ohe = ohes[i]
    #         res_ohes.append(ohe)
    #     features_df[feature] = ohe.transform(features_df[feature])
    cat_cols = ['category', 'host', 'netloc']
    if not ohe:
        ohe = OneHotEncoder()
        ohe.fit(features_df[cat_cols])
    features_df = pd.concat([features_df, pd.DataFrame(ohe.transform(features_df[cat_cols]).toarray())], axis=1)
     
    # res_scalers = ['title_len', 'body_len', 'answer_len', 'bert_title_len', 'bert_body_len', 'bert_answer_len']
    # for i, feature in enumerate([]):
    #     if not scalers:
    #         scaler = StandardScaler()
    #         scaler.fit(features_df[feature])
    #         res_scalers.append(scaler)
    #     else:
    #         scaler = scalers[i]
    #         res_scalers.append(scaler)
    #     features_df[feature] = scaler.transform(features_df[feature])

    num_cols = ['title_len', 'body_len', 'answer_len', 'bert_title_len', 'bert_body_len', 'bert_answer_len']
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(features_df[num_cols])
    features_df.loc[:, num_cols] = scaler.transform(features_df[num_cols])
    
    bert_df = pd.DataFrame(bert_preds).add_prefix('bert_')
    roberta_df = pd.DataFrame(roberta_preds).add_prefix('roberta_')
    xlnet_df = pd.DataFrame(xlnet_preds).add_prefix('xlnet_')
    
    features_df = pd.concat([features_df, bert_df, roberta_df, xlnet_df], axis=1)
    
    return features_df, ohe, scaler

In [306]:
labels = [
    'question_asker_intent_understanding',
    'question_body_critical',
    'question_conversational',
    'question_expect_short_answer',
    'question_fact_seeking',
    'question_has_commonly_accepted_answer',
    'question_interestingness_others',
    'question_interestingness_self',
    'question_multi_intent',
    'question_not_really_a_question',
    'question_opinion_seeking',
    'question_type_choice',
    'question_type_compare',
    'question_type_consequence',
    'question_type_definition',
    'question_type_entity',
    'question_type_instructions',
    'question_type_procedure',
    'question_type_reason_explanation',
    'question_type_spelling',
    'question_well_written',
    'answer_helpful',
    'answer_level_of_information',
    'answer_plausible',
    'answer_relevance',
    'answer_satisfaction',
    'answer_type_instructions',
    'answer_type_procedure',
    'answer_type_reason_explanation',
    'answer_well_written'
]

In [310]:
# %debug
features_df, ohes, scalers = mk_features(trn_df, bert_y_preds, roberta_y_preds, xlnet_y_preds, '../mnt/checkpoints/e078/datasets/', ohe=None, scaler=None, labels=labels)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

100%|██████████| 6079/6079 [00:39<00:00, 155.62it/s]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

 51%|█████     | 3090/6079 [00:20<00:19, 151.43it/s]


100%|██████████| 6079/6079 [00:38<00:00, 157.65it/s]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [311]:
features_df

,qa_id,category,fold,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written,host,netloc,title_len,body_len,answer_len,bert_title_len,bert_body_len,bert_answer_len,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,bert_0,bert_1,bert_2,bert_3,bert_4,bert_5,bert_6,bert_7,bert_8,bert_9,bert_10,bert_11,bert_12,bert_13,bert_14,bert_15,bert_16,bert_17,bert_18,bert_19,bert_20,bert_21,bert_22,bert_23,bert_24,bert_25,bert_26,bert_27,bert_28,bert_29,bert_30,bert_31,bert_32,bert_33,bert_34,bert_35,bert_36,bert_37,bert_38,bert_39,bert_40,bert_41,bert_42,bert_43,bert_44,bert_45,bert_46,bert_47,bert_48,bert_49,bert_50,bert_51,bert_52,bert_53,bert_54,bert_55,bert_56,bert_57,bert_58,bert_59,roberta_0,roberta_1,roberta_2,roberta_3,roberta_4,roberta_5,roberta_6,roberta_7,roberta_8,roberta_9,roberta_10,roberta_11,roberta_12,roberta_13,roberta_14,roberta_15,roberta_16,roberta_17,roberta_18,roberta_19,roberta_20,roberta_21,roberta_22,roberta_23,roberta_24,roberta_25,roberta_26,roberta_27,roberta_28,roberta_29,roberta_30,roberta_31,roberta_32,roberta_33,roberta_34,roberta_35,roberta_36,roberta_37,roberta_38,roberta_39,roberta_40,roberta_41,roberta_42,roberta_43,roberta_44,roberta_45,roberta_46,roberta_47,roberta_48,roberta_49,roberta_50,roberta_51,roberta_52,roberta_53,roberta_54,roberta_55,roberta_56,roberta_57,roberta_58,roberta_59,xlnet_0,xlnet_1,xlnet_2,xlnet_3,xlnet_4,xlnet_5,xlnet_6,xlnet_7,xlnet_8,xlnet_9,xlnet_10,xlnet_11,xlnet_12,xlnet_13,xlnet_14,xlnet_15,xlnet_16,xlnet_17,xlnet_18,xlnet_19,xlnet_20,xlnet_21,xlnet_22,xlnet_23,xlnet_24,xlnet_25,xlnet_26,xlnet_27,xlnet_28,xlnet_29,xlnet_30,xlnet_31,xlnet_32,xlnet_33,xlnet_34,xlnet_35,xlnet_36,xlnet_37,xlnet_38,xlnet_39,xlnet_40,xlnet_41,xlnet_42,xlnet_43,xlnet_44,xlnet_45,xlnet_46,xlnet_47,xlnet_48,xlnet_49,xlnet_50,xlnet_51,xlnet_52,xlnet_53,xlnet_54,xlnet_55,xlnet_56,xlnet_57,xlnet_58,xlnet_59
0,0,LIFE_ARTS,0,1.000000,0.333333,0.000000,0.000000,0.000000,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.000000,0.000000,0.000000,1.000000,host_stackexchange,photo,1.050273,0.135656,0.109141,0.317333,-0.120117,-0.067228,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.937834,0.633688,0.054328,0.559676,0.682502,0.403668,0.635739,0.591393,0.394091,0.004519,0.719388,0.044919,0.129137,0.026241,0.005142,0.123739,0.393864,0.079044,0.507497,0.002152,0.921155,0.978332,0.690518,0.990210,0.988864,0.925071,0.084333,0.060422,0.917059,0.938186,0.948040,0.693285,0.052300,0.539415,0.674708,0.225477,0.681708,0.597191,0.194366,0.005146,0.792736,0.041187,0.077567,0.026021,0.004593,0.065753,0.361647,0.067497,0.510446,0.001913,0.948859,0.970516,0.714862,0.986803,0.985297,0.923511,0.126159,0.059128,0.868586,0.936540,0.939356,0.690926,0.140137,0.456486,0.695388,0.427599,0.675041,0.654015,0.326620,0.005562,0.802528,0.071001,0.165792,0.025142,0.006687,0.034890,0.074643,0.

In [318]:
drop_cols = ['qa_id', 'category', 'fold', 'host', 'netloc']

reses = []
for fold in range(5):
    trn_x = features_df.query(f'fold != {fold}').drop(drop_cols+labels, axis=1).values
    trn_y = features_df.query(f'fold != {fold}')[labels].values
    val_x = features_df.query(f'fold == {fold}').drop(drop_cols+labels, axis=1).values
    val_y = features_df.query(f'fold == {fold}')[labels].values
    
    model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
    model.fit(trn_x, trn_y)
    y_pred = model.predict(val_x)
    res = compute_spearmanr(val_y, y_pred)
    print(np.mean(res))
    reses.append(res)
print(np.mean(reses))
# model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
# model.fit(features_df.drop(drop_cols+labels, axis=1).values)
# model.predict

0.41041107804619154
0.40336732085623583
0.4204717163820624
0.407737655696662
0.39578742543838225


In [319]:
np.mean([0.41041107804619154,
0.40336732085623583,
0.4204717163820624,
0.407737655696662,
0.39578742543838225])

0.4075550392839068

In [333]:
drop_cols = ['qa_id', 'category', 'fold', 'host', 'netloc']

reses = []
for fold in range(5):
    trn_x = features_df.query(f'fold != {fold}').drop(drop_cols+labels, axis=1).values
    trn_y = features_df.query(f'fold != {fold}')[labels].values
    val_x = features_df.query(f'fold == {fold}').drop(drop_cols+labels, axis=1).values
    val_y = features_df.query(f'fold == {fold}')[labels].values
    
    model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
    model.fit(trn_x, trn_y)
    y_pred = model.predict(val_x)
    res = compute_spearmanr(val_y, y_pred)
    print(np.mean(res))
    reses.append(res)
print(np.mean(reses))
# model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
# model.fit(features_df.drop(drop_cols+labels, axis=1).values)
# model.predict

0.39832241941874347
0.39039939293439097
0.41126443142167607
0.39548347638047676
0.3887355061777134
0.39684104526660013


In [322]:
model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
model.fit(features_df.drop(drop_cols+labels, axis=1).values, features_df[labels].values)
y_pred = model.predict(features_df.drop(drop_cols+labels, axis=1).values)
res = compute_spearmanr(features_df[labels].values, y_pred)

In [323]:
np.mean(res)

0.44343593128277276

In [329]:
drop_cols = ['qa_id', 'category', 'fold', 'host', 'netloc']

reses = []
for fold in range(5):
    trn_x = features_df.query(f'fold != {fold}').drop(drop_cols+labels, axis=1).values
    trn_y = features_df.query(f'fold != {fold}')[labels].values
    val_x = features_df.query(f'fold == {fold}').drop(drop_cols+labels, axis=1).values
    val_y = features_df.query(f'fold == {fold}')[labels].values
    
    model = Lasso(alpha=0.0001, random_state=42)
    model.fit(trn_x, trn_y)
    y_pred = model.predict(val_x)
    res = compute_spearmanr(val_y, y_pred)
    print(np.mean(res))
    reses.append(res)
print(np.mean(reses))
# model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
# model.fit(features_df.drop(drop_cols+labels, axis=1).values)
# model.predict

0.41087039287624344
0.4058923725171438
0.4209248201796327
0.40854678227492225
0.3995903393449829
0.40916494143858506


In [443]:
drop_cols = ['qa_id', 'category', 'fold', 'host', 'netloc']

reses = []
for fold in range(5):
    trn_x = features_df.query(f'fold != {fold}').drop(drop_cols+labels, axis=1).values
    trn_y = features_df.query(f'fold != {fold}')[labels].values
    val_x = features_df.query(f'fold == {fold}').drop(drop_cols+labels, axis=1).values
    val_y = features_df.query(f'fold == {fold}')[labels].values
    
    model = OneVsRestClassifier(LogisticRegression(C=0.01, random_state=42, multi_class='ovr'))
    model.fit(trn_x, trn_y)
    y_pred = model.predict(val_x)
    res = compute_spearmanr(val_y, y_pred)
    print(np.mean(res))
    reses.append(res)
print(np.mean(reses))
# model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
# model.fit(features_df.drop(drop_cols+labels, axis=1).values)
# model.predict

ValueError: Multioutput target data is not supported with label binarization

In [336]:
model = Lasso(alpha=0.0001, random_state=42)
model.fit(features_df.drop(drop_cols+labels, axis=1).values, features_df[labels].values)
y_pred = model.predict(features_df.drop(drop_cols+labels, axis=1).values)
res = compute_spearmanr(features_df[labels].values, y_pred)

In [337]:
np.mean(res)

0.4432701170724508

## 試しに whole data train の stacking やってみる
 - ※ 2 : 1 : 1
 - 2 は avg

In [402]:
# %debug
bert_y_trues, bert_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(bert_question_dict, bert_answer_dict, avg=True)
roberta_y_trues, roberta_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(roberta_question_dict, roberta_answer_dict, avg=True)
xlnet_y_trues, xlnet_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(xlnet_question_dict, xlnet_answer_dict, avg=True)

In [404]:
# %debug
single_bert_y_trues, single_bert_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(bert_question_dict, bert_answer_dict, single=True, avg=False)
single_roberta_y_trues, single_roberta_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(roberta_question_dict, roberta_answer_dict, single=True, avg=False)
single_xlnet_y_trues, single_xlnet_y_preds = get_y_trues_and_y_preds_from_QA_snapshota_dicts(xlnet_question_dict, xlnet_answer_dict, single=True, avg=False)

In [405]:
trn_df = pd.read_csv('../mnt/inputs/origin/train.csv')
tst_df = pd.read_csv('../mnt/inputs/origin/test.csv')

In [406]:
from sklearn.model_selection import GroupKFold as GKF

gkf = GKF(5)
fold = list(gkf.split(trn_df.qa_id, groups=trn_df.question_body))

In [407]:
val_idxes = [i[1] for i in fold]
val_idx = np.concatenate(val_idxes)

In [408]:
trn_df['fold'] = None
for i, _val_idx in enumerate(val_idxes):
    trn_df.loc[_val_idx, 'fold'] = i

In [409]:
trn_df['fold'].fillna(-1).value_counts()

3    1216
2    1216
1    1216
0    1216
4    1215
Name: fold, dtype: int64

In [410]:
(trn_df.iloc[val_idx].iloc[:, 11:-1].values == bert_y_trues[:, :30]).all()

True

In [411]:
trn_df = trn_df.iloc[val_idx].reset_index(drop=True)

In [412]:
# 置換完了
(trn_df.iloc[:, 11:-1].values == bert_y_trues[:, :30]).all()

True

In [413]:
# %debug
features_df, ohes, scalers = mk_features(trn_df, bert_y_preds, single_roberta_y_preds, single_xlnet_y_preds, '../mnt/checkpoints/e078/datasets/', ohe=None, scaler=None, labels=labels)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

100%|██████████| 6079/6079 [00:37<00:00, 161.82it/s]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

 57%|█████▋    | 3460/6079 [00:21<00:16, 162.05it/s]


100%|██████████| 6079/6079 [00:37<00:00, 160.65it/s]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [423]:
bert_y_preds.shape

(6079, 30)

In [414]:
tst_df['fold'] = -1
tst_features, _, _ = mk_features(tst_df, bert_y_preds, single_roberta_y_preds, single_xlnet_y_preds, '../mnt/checkpoints/e078/datasets/', ohe=ohes, scaler=scalers, labels=[])

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [424]:
model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
model.fit(features_df.drop(drop_cols+labels, axis=1).values, features_df[labels].values)
y_pred = model.predict(features_df.drop(drop_cols+labels, axis=1).values)
res = compute_spearmanr(features_df[labels].values, y_pred)
np.mean(res)

0.438265342954586

In [425]:
eval_scores, opt_eval_scores, optRs = blend_and_evaluate(features_df[labels].values, [y_pred, ], compute_spearmanr)

original_score: 0.438265342954586
opt_score: 0.4881796345764086


In [426]:
eval_scores

[0.41753112199777104,
 0.6764953319096272,
 0.42041124578475975,
 0.3445646646410174,
 0.38704381965286866,
 0.45130652167676083,
 0.3961194908046353,
 0.529847909439801,
 0.6119612323931802,
 0.12516425938870868,
 0.510398186761741,
 0.7605434726794571,
 0.37125442681495224,
 0.19413188857848201,
 0.35777555752374846,
 0.46217239384555947,
 0.7923505527480342,
 0.3938071010494091,
 0.6946415350604473,
 0.07257527239457269,
 0.5285877260802457,
 0.2948007237476026,
 0.4851563812807026,
 0.21496838861715717,
 0.22525026989531646,
 0.38443902552982523,
 0.7721339186049994,
 0.332985963069032,
 0.7071401193763913,
 0.2324017872907762]

In [427]:
opt_eval_scores

[0.41343022683327774,
 0.6720030412486051,
 0.4938149372930572,
 0.3391910490029068,
 0.39187799667690215,
 0.5072403885501177,
 0.39808415086767984,
 0.5371936935309224,
 0.6192335166138851,
 0.18109855772262892,
 0.4967482171718512,
 0.772318523480993,
 0.5911052329202999,
 0.33529408603749244,
 0.659875548303857,
 0.6177865992485571,
 0.79790056640927,
 0.3745143276058302,
 0.6894973122086349,
 0.6151022328164457,
 0.5296435154397106,
 0.29602617927826935,
 0.4834727079718044,
 0.21961170063223812,
 0.22664964858647108,
 0.3847117607697327,
 0.7680312871946515,
 0.3033595906063066,
 0.6962289343060919,
 0.23434350796376646]

In [428]:
nes_dict = {
    'model': model, 
    'ohe': ohes,
    'scaler': scalers,
}

In [429]:
with open('../mnt/datasets/stackings/e10X_stacking/e10X_stacking_dict.pkl', 'wb') as fout:
    pickle.dump(nes_dict, fout)

In [430]:
with open('../mnt/datasets/stackings/e10X_stacking/optRs.pkl', 'wb') as fout:
    pickle.dump(optRs, fout)

#### ちなみに...？

In [421]:
drop_cols = ['qa_id', 'category', 'fold', 'host', 'netloc']

reses = []
temp_y_preds = []
temp_y_trues = []
for fold in range(5):
    trn_x = features_df.query(f'fold != {fold}').drop(drop_cols+labels, axis=1).values
    trn_y = features_df.query(f'fold != {fold}')[labels].values
    val_x = features_df.query(f'fold == {fold}').drop(drop_cols+labels, axis=1).values
    val_y = features_df.query(f'fold == {fold}')[labels].values
    
    model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
    model.fit(trn_x, trn_y)
    y_pred = model.predict(val_x)
    res = compute_spearmanr(val_y, y_pred)
    temp_y_preds.append(y_pred)
    temp_y_trues.append(val_y)
    print(np.mean(res))
    reses.append(res)
print(np.mean(reses))
# model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
# model.fit(features_df.drop(drop_cols+labels, axis=1).values)
# model.predict

0.40981280341747495
0.40372851673832894
0.4206026783853577
0.40815010880027536
0.39626454923180715
0.4077117313146489


In [422]:
_, _, _ = blend_and_evaluate(np.concatenate(temp_y_trues, axis=0), [np.concatenate(temp_y_preds, axis=0), ], compute_spearmanr)

/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:2525: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


original_score: 0.4053620624917785
opt_score: 0.4309404384262748


In [432]:
tst_features.shape, features_df.shape

((6079, 168), (6079, 198))

In [434]:
features_df.drop(drop_cols+labels, axis=1).columns.tolist()

['title_len',
 'body_len',
 'answer_len',
 'bert_title_len',
 'bert_body_len',
 'bert_answer_len',
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 'bert_0',
 'bert_1',
 'bert_2',
 'bert_3',
 'bert_4',
 'bert_5',
 'bert_6',
 'bert_7',
 'bert_8',
 'bert_9',
 'bert_10',
 'bert_11',
 'bert_12',
 'bert_13',
 'bert_14',
 'bert_15',
 'bert_16',
 'bert_17',
 'bert_18',
 'bert_19',
 'bert_20',
 'bert_21',
 'bert_22',
 'bert_23',
 'bert_24',
 'bert_25',
 'bert_26',
 'bert_27',
 'bert_28',
 'bert_29',
 'roberta_0',
 'roberta_1',
 'roberta_2',
 'roberta_3',
 'roberta_4',
 'roberta_5',
 'roberta_6',
 'roberta_7',
 'roberta_8',
 'roberta_9',
 'roberta_10',
 'roberta_11',
 'roberta_12',
 'roberta_13',
 'roberta_14',
 'roberta_1

## 混ぜてみる
 - 4 つ全て混ぜてみる
 - 3/4 の　snapshot 混ぜてみる？

#### 試しに single, top-2 snapshots を見てみる

In [340]:
blend_and_evaluate(single_bert_y_trues, [single_bert_y_preds,  ], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [single_roberta_y_preds, ], compute_spearmanr)
# blend_and_evaluate(single_bert_y_trues, [single_gpt2_y_preds, ], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [single_xlnet_y_preds], compute_spearmanr)
print('fini!')

original_score: 0.39658756843296844
opt_score: 0.4240074936181189


/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:2525: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


original_score: 0.3952086168898242
opt_score: 0.417814664868746
original_score: 0.39510187276130454
opt_score: 0.42243226674838147
fini!


In [341]:
blend_and_evaluate(single_bert_y_trues, [bert_y_preds,  ], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [roberta_y_preds, ], compute_spearmanr)
# blend_and_evaluate(single_bert_y_trues, [gpt2_y_preds, ], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [xlnet_y_preds], compute_spearmanr)
print('fini!')

original_score: 0.39658756843296844
opt_score: 0.4240074936181189
original_score: 0.3952086168898242
opt_score: 0.417814664868746
original_score: 0.39510187276130454
opt_score: 0.42243226674838147
fini!


#### all blends がどうなるかを見てみる

In [343]:
# eval_scores, opt_eval_scores = blend_and_evaluate(single_bert_y_trues, [single_bert_y_preds,  single_roberta_y_preds, single_gpt2_y_preds, single_xlnet_y_preds], compute_spearmanr)
blend_and_evaluate(single_bert_y_trues, [single_bert_y_preds,  single_roberta_y_preds, single_xlnet_y_preds], compute_spearmanr)
print('fini!')

original_score: 0.416481507046034
opt_score: 0.4491590833618151
fini!


## 2, 2, 1 じゃ disk 足りなかった...

In [20]:
blend_and_evaluate(bert_y_trues, [single_bert_y_preds,  single_roberta_y_preds, xlnet_y_preds], compute_spearmanr)
blend_and_evaluate(bert_y_trues, [bert_y_preds,  single_roberta_y_preds, single_xlnet_y_preds], compute_spearmanr)
blend_and_evaluate(bert_y_trues, [single_bert_y_preds,  roberta_y_preds, single_xlnet_y_preds], compute_spearmanr)
print('fini!')

original_score: 0.41745646679452425
opt_score: 0.4494514724953275
original_score: 0.41786576946688975
opt_score: 0.4513792627190148
original_score: 0.4181574464924081
opt_score: 0.44796109262403244
fini!


#### 2, 1, 1 で行くことにするので optR を取得

In [21]:
eval_scores, opt_eval_scores, optRs = blend_and_evaluate(bert_y_trues, [bert_y_preds,  single_roberta_y_preds, single_xlnet_y_preds], compute_spearmanr)

original_score: 0.41786576946688975
opt_score: 0.4513792627190148


In [23]:
!mkdir ../mnt/datasets/sub_e078_e083_bert2_roberta1_xlnet1

In [24]:
with open('../mnt/datasets/sub_e078_e083_bert2_roberta1_xlnet1/optRs.pkl', 'wb') as fout:
    pickle.dump(optRs, fout)

In [27]:
','.join(([str(i) for i in eval_scores]))

'0.3888662513004357,0.6264747430024324,0.417198052103926,0.31356995786700836,0.36621009889416495,0.43161328159188245,0.35960846883953307,0.5044099886093536,0.604524708512999,0.09185984413637717,0.4871602561126112,0.7588073461879603,0.3687724113728429,0.1878256807431471,0.3623045162839472,0.46221497917924714,0.7845698434998972,0.37561304685622554,0.685135875113699,0.06569265158283247,0.509277164422869,0.26846500559030767,0.44030220154571237,0.17059028253021058,0.18652642117523097,0.35943031009217935,0.7627075286902857,0.28993040895144306,0.6928979913635044,0.21341376785442578'

In [28]:
','.join(([str(i) for i in opt_eval_scores]))

'0.38647035222851883,0.6251840213259875,0.503341694761477,0.3140683242310781,0.37238351698553224,0.48891838338012195,0.3688643054450728,0.5174029320574922,0.6075334223395167,0.11234184006252432,0.4734302164363926,0.7724553926978851,0.5679733516339525,0.3086234975886461,0.6273452824025416,0.6104216226094872,0.7913164255678906,0.35227648410197876,0.6743979217445293,0.21251606433106468,0.5092991566008891,0.25989741044634646,0.4419292131037434,0.172812412462297,0.1891229575649494,0.360249882795264,0.7579315945321048,0.2690066667578972,0.6818714797543032,0.21199205562095982'

In [84]:
','.join([str(i) for i in eval_scores])

'0.38753764855932404,0.624591236065633,0.41612521851465645,0.31631905154404777,0.36958084656834655,0.4333397143525736,0.35647225646179814,0.5053740180720495,0.6024717179943407,0.09343879770080012,0.48788970013367006,0.7554768784733894,0.37043040207462113,0.19175643232185877,0.3632398123928754,0.4644149810472098,0.7882689049397114,0.37508934129137833,0.6875082798335962,0.06859634383678749,0.5058311174919892,0.26776035706473816,0.42468236885314736,0.18264774434316508,0.1960057923938459,0.3571316272381341,0.7629198053679217,0.29516358912502183,0.6885185616941287,0.1986028427408061'

In [85]:
','.join([str(i) for i in opt_eval_scores])

'0.38610697100222874,0.6220378810356756,0.4803108598080848,0.3087254707253665,0.3767010478726585,0.48153256901696284,0.3623044669561681,0.5109740248841105,0.6079582135843575,0.10998964284517494,0.4870548021086867,0.7644745305398798,0.5468575104750577,0.2850693099375624,0.6439619945737882,0.6181433940398664,0.7950887499733366,0.3480114972488913,0.6853066014210485,0.36239255378317503,0.503890831161848,0.26595541365451547,0.4260248516788567,0.18501720906281013,0.20022371905506964,0.3569060052051175,0.7579715572197582,0.27970705751496816,0.6821275873309829,0.20174712128915573'

#### weights どうすればよいか見てみる

In [86]:
np.arange(0, 1.1, 0.1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [ ]:
for i in np.arange(0, 1.1, 0.1):
    for j in np.arange(0, 1.01- i, 0.1):
        weights = [i, j, 1-i-j]
        print(f' ---------- {weights} -----------')
        eval_scores, opt_eval_scores, optRs = blend_and_evaluate(bert_y_trues,
                                                                                                                 [single_bert_y_preds,  roberta_y_preds, single_xlnet_y_preds],
                                                                                                                 compute_spearmanr,
                                                                                                                 weights=weights)

 ---------- [0.0, 0.0, 1.0] -----------
original_score: 0.3906094115371919
opt_score: 0.4208016538440192
 ---------- [0.0, 0.1, 0.9] -----------
original_score: 0.39690428620399504
opt_score: 0.42660201555875216
 ---------- [0.0, 0.2, 0.8] -----------
original_score: 0.40194251860786706
opt_score: 0.4322132022423592
 ---------- [0.0, 0.30000000000000004, 0.7] -----------
original_score: 0.4058057699680523
opt_score: 0.4386623102560342
 ---------- [0.0, 0.4, 0.6] -----------
original_score: 0.40850785092762126
opt_score: 0.44218874682162584
 ---------- [0.0, 0.5, 0.5] -----------
original_score: 0.41002182412980964
opt_score: 0.4462993778180665
 ---------- [0.0, 0.6000000000000001, 0.3999999999999999] -----------
original_score: 0.41026148164290954
opt_score: 0.4472571871083271
 ---------- [0.0, 0.7000000000000001, 0.29999999999999993] -----------
original_score: 0.4092935254165069
opt_score: 0.4447945948297066
 ---------- [0.0, 0.8, 0.19999999999999996] -----------
original_score: 0.40

## まとめ
 - (ちょっと気に入らんけど) bert * 2, roberta * 2, xlnet * 1 の average が良さそう (* 1, * 1, * 1, * 1 よりも)
 - 多分もう一つ別の良いモデルを入れられれば伸びる